In [4]:
import pandas as pd
from ortools.linear_solver import pywraplp

# Load Excel file
super_data = {}
datatype = ["Dispatch", "Trip Rates", "Vehicle Dimensions"]
for i in datatype:
    super_data[i] = pd.read_excel("Per Trip Data.xlsx", sheet_name=i)

loads_df = super_data['Dispatch']  # Load details
rates_df = super_data['Trip Rates']  # Rates details

# Sort the rates for binary search
rates_df = rates_df.sort_values(by=["Origin City", "Origin State", "Destination City", "Destination State"]).reset_index(drop=True)

# Convert sorted into a list
sorted_od_pairs = list(zip(rates_df["Origin City"], rates_df["Origin State"], rates_df["Destination City"], rates_df["Destination State"]))

# List of vehicle type columns (all columns except location details)
vehicle_columns = [col for col in rates_df.columns if col not in ["Origin City", "Origin State", "Destination City", "Destination State"]]

# Binary search to find index of a given (WH City, WH State, Destination, Destination State)
def binary_search(origin_city, origin_state, dest_city, dest_state):
    left, right = 0, len(sorted_od_pairs) - 1
    while left <= right:
        mid = (left + right) // 2
        if sorted_od_pairs[mid] == (origin_city, origin_state, dest_city, dest_state):
            return mid  # Found the index
        elif sorted_od_pairs[mid] < (origin_city, origin_state, dest_city, dest_state):
            left = mid + 1
        else:
            right = mid - 1
    return -1  # Not found

# Function to find rates for all vehicles using binary search
def find_rates(origin_city, origin_state, dest_city, dest_state):
    idx = binary_search(origin_city, origin_state, dest_city, dest_state)
    if idx != -1:
        return rates_df.iloc[idx][vehicle_columns].to_dict()  # Return all vehicle rates as a dictionary
    return {vehicle: None for vehicle in vehicle_columns}  # Return None for all vehicles if no match

# Apply binary search to map rates
rate_mappings = loads_df.apply(lambda row: find_rates(row["WH City"], row["WH State"], row["Destination"], row["Destination State"]), axis=1)

# Convert dictionary results into separate columns for each vehicle
rate_mappings_df = pd.DataFrame(rate_mappings.tolist())

# Merge the mapped rates back into the loads DataFrame
loads_df = pd.concat([loads_df, rate_mappings_df], axis=1)

def get_data(data, lane, vehicle_type):
    oc, dc, ds = lane
    try:
        row = data.loc[(data['WH City'] == oc) & (data['Destination'] == dc) & (data['Destination State'] == ds)]
        rate = row[vehicle_type].values[0]
        total_weight = row['Weight'].values[0]
        total_cft = row['CFT'].values[0]
        quantity = row['Qty'].values[0]
        dealer_count = row['Dealer Count'].values[0]
        ptl_charge = row['PTL Charges'].values[0]

        if quantity == 0:
            return "Quantity is zero, cannot compute unit weight"

        return {
            'rate': rate,
            'unit_weight': total_weight / quantity,
            'unit_cft': total_cft / quantity,
            'qty': quantity,
            'dealers': dealer_count,
            'ptl_charge': ptl_charge
        }

    except IndexError:
        return f"No data for {lane}"
    except ZeroDivisionError:
        return "Quantity is zero, cannot compute unit weight"

def transportation_model(loads_df, super_data, output_file="transportation_results.xlsx"):
    max_drops = int(input("Enter the maximum number of drops per trip: "))
    extra_cost = int(input("Enter the extra drop cost: "))
    Optimized_Cost = 0
    overall_total_cost = 0
    data = loads_df
    vehicle_capacity = {}

    for col in super_data['Vehicle Dimensions'].columns[1:]:
        vehicle_capacity[col] = {
            'Weight': super_data['Vehicle Dimensions'].loc[3, col],
            'CFT': super_data['Vehicle Dimensions'].loc[2, col]
        }
    vehicle_types = list(vehicle_capacity.keys())

    unique_dates = data["Date"].unique()
    results = []

    for date in unique_dates:
        datewise_data = data[data["Date"] == date]
        lanes = list(datewise_data[['WH City', 'Destination', 'Destination State']].itertuples(index=False, name=None))

        max_drops_per_trip = max_drops
        extra_drop_cost = extra_cost

        solver = pywraplp.Solver.CreateSolver("SCIP")

        # Decision variables
        x = {}
        for lane in lanes:
            for veh in vehicle_types:
                x[(lane, veh)] = solver.IntVar(0, solver.infinity(), f'x[{lane},{veh}]')

        extra_drops = {}
        for lane in lanes:
            extra_drops[lane] = solver.IntVar(0, solver.infinity(), f'extra_drops[{lane}]')

        # Minimize Cost (Including Extra Drops)
        solver.Minimize(
            solver.Sum(
                x[(lane, veh)] * get_data(datewise_data, lane, veh)['rate']
                for lane in lanes
                for veh in vehicle_types
            ) +
            solver.Sum(extra_drops[lane] * extra_drop_cost for lane in lanes)
        )

        # Constraints
        for lane in lanes:
            lane_data = get_data(datewise_data, lane, vehicle_types[0])
            demand_units = lane_data['qty']
            unit_cft = lane_data['unit_cft']
            unit_weight = lane_data['unit_weight']

            solver.Add(
                solver.Sum(
                    x[(lane, veh)] * (vehicle_capacity[veh]['CFT'] // unit_cft if unit_cft > 0 else 9999)
                    for veh in vehicle_types
                ) >= demand_units
            )
            solver.Add(
                solver.Sum(
                    x[(lane, veh)] * (vehicle_capacity[veh]['Weight'] // unit_weight if unit_weight > 0 else 9999)
                    for veh in vehicle_types
                ) >= demand_units
            )

        for lane in lanes:
            lane_data = get_data(datewise_data, lane, vehicle_types[0])
            total_vehicles = solver.Sum(x[(lane, veh)] for veh in vehicle_types)

            solver.Add(extra_drops[lane] >= lane_data['dealers'] - total_vehicles)
            solver.Add(max_drops_per_trip * total_vehicles >= lane_data['dealers'])

        # Solve the model
        status = solver.Solve()

        # Collect results
        if status == pywraplp.Solver.OPTIMAL:
            date_total_cost = solver.Objective().Value()
            overall_total_cost += date_total_cost

            for lane in lanes:
                lane_vehicles = {veh: int(x[(lane, veh)].solution_value()) for veh in vehicle_types if x[(lane, veh)].solution_value() > 0}
                lane_cost = sum(x[(lane, veh)].solution_value() * get_data(datewise_data, lane, veh)['rate'] for veh in vehicle_types)
                extra_drop_count = int(extra_drops[lane].solution_value())
                extra_drop_charge = extra_drop_count * extra_drop_cost
                ptl_charge = get_data(datewise_data, lane, vehicle_types[0])['ptl_charge']
                Optimal_cost = min(lane_cost + extra_drop_charge, ptl_charge)
                Optimized_Cost += Optimal_cost

                # Create row for the results
                result_row = {
                    "Lane": f"{lane[0]} -> {lane[1]}, {lane[2]}",
                    "Date": date,
                    "FTL Cost": lane_cost + extra_drop_charge,
                    "Extra Drop Charges": extra_drop_charge,
                    "PTL Cost": ptl_charge,
                    "Optimum Cost": Optimal_cost,
                    "Mode": "FTL" if lane_cost + extra_drop_charge < ptl_charge else "PTL"
                }

                # Add vehicle types
                for veh in vehicle_types:
                    result_row[veh] = lane_vehicles.get(veh, 0)

                results.append(result_row)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Save to Excel
    results_df.to_excel(output_file, index=False)

    print("\nResults saved to:", output_file)
    print("Total Transportation Cost:", Optimized_Cost)

transportation_model(loads_df, super_data)


Enter the maximum number of drops per trip: 3
Enter the extra drop cost: 1000

Results saved to: transportation_results.xlsx
Total Transportation Cost: 248977638.97500008
